In [90]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

In [91]:
driver = webdriver.Chrome('C:/Users/daniel/chromedriver.exe')

#open the webpage
driver.get("http://www.instagram.com")

#target username
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

#enter username and password
username.clear()
username.send_keys("@username")
password.clear()
password.send_keys("password")

#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

In [92]:
#target the search input field
searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Search']")))
searchbox.clear()

#search for the hashtag cat
keyword = "#IndonesiainJapan"
searchbox.send_keys(keyword)

driver.get("http://www.instagram.com/explore/tags/IndonesiainJapan/")

#FIXING THE DOUBLE ENTER
#time.sleep(5) # Wait for 5 seconds
#my_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, '/" + keyword[1:] + "/')]")))
#my_link.click()

In [93]:
n_scrolls = 1
scrolls_to = 20
anchors = []
while n_scrolls != scrolls_to:
    driver.get("http://www.instagram.com/explore/tags/IndonesiainJapan/")
    for j in range(0, n_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
    p = driver.find_elements_by_xpath('//div[@class="v1Nh3 kIKUG  _bz0w"]')
    for e in p:
        link = e.find_element_by_xpath('.//a').get_attribute("href")
        anchors.append(link)
    n_scrolls = n_scrolls+1

In [72]:
# coba 1
#anchors = []
#p = driver.find_elements_by_xpath('//div[@class="v1Nh3 kIKUG  _bz0w"]')
#for e in p:
#    link = e.find_element_by_xpath('.//a').get_attribute("href")
#    anchors.append(link)

In [94]:
len(anchors)

834

In [97]:
import numpy as np
x = np.array(anchors)
anchors = np.unique(x)

In [46]:
# coba 2
#anchors = driver.find_elements_by_tag_name('a')
#anchors = [a.get_attribute('href') for a in anchors]
#narrow down all links to image links only
#anchors = [a for a in anchors if str(a).startswith("https://www.instagram.com/p/")]

#print('Found ' + str(len(anchors)) + ' links to images')

In [101]:
akun = []
caption = []
hashtag = []
waktu = []
for anc in anchors:
    driver.get(anc)
    source = driver.page_source
    data = bs(source, 'html.parser')
    body = data.find('body')
    akun.append(body.find("div", {"class": "C4VMK"}).find('h2').text)
    caption.append(body.find("div", {"class": "C4VMK"}).find("span", {"class": ""}).text)
    waktu.append(body.find("div", {"class": "C4VMK"}).find("time")['datetime'])
    cap = body.find("div", {"class": "C4VMK"}).find("span", {"class": ""}).findAll('a')
    tag = ' '
    for i in cap:
        tag = tag + i.text + ' '
    hashtag.append(tag)

In [105]:
import pandas as pd
df = pd.DataFrame({
    'alamat' : anchors,
    'nama' : akun,
    "text": caption,
    'hashtag' : hashtag,
    'waktu_post' : waktu
})

In [106]:
df.to_excel("output.xlsx")